In [1]:
import numpy as np
from collections import defaultdict
import nltk 
import pandas as pd
import os 
import json
import io
import gzip

The code shown below is from 

In [14]:
# read in a gz file
def readGz(fname):
    gz = gzip.open(fname, 'rb')
    f = io.BufferedReader(gz)
    data = []
    rows = 0

    for l in f.readlines():
        rows = rows + 1
        data.append(eval(l))
        if rows == 10000:
            break;
        
    gz.close()
    return data
#     for l in gzip.open(f):
#         yield eval(l)

In [15]:
%%time
#parsing in the data, takes time
places = readGz("places.clean.json.gz")
users = readGz("users.clean.json.gz")
reviews = readGz("reviews.clean.json.gz")

Wall time: 3min 22s


In [16]:
places = pd.DataFrame(places)
users = pd.DataFrame(users)
reviews = pd.DataFrame(reviews)